Prapare Training and testing set


In [ ]:
import os
import shutil

folder_name = 'Summer'
#folder_name = 'Winter'
#folder_name = 'Fall'
#folder_name = 'Spring'
main_folder = 'drive/MyDrive/Thesis/Nordland/' + folder_name
target_folder = 'drive/MyDrive/Thesis/Nordland/training_10'
#target_folder = 'drive/MyDrive/Thesis/Nordland/testing_10'

# Get all file names in the main folder
files = os.listdir(main_folder)

def numerical_sort(value):
    parts = value.split('.')
    return int(parts[0])

# Sort the files for consistency
files = sorted(files, key=numerical_sort)

# Calculate the number of files per label for train and test sets
num_same_place = 10

place_number = 1
total_count = 1
label = 'place' + str(place_number)

# Loop over the files and move them to the train and test folders
for i, file in enumerate(files):
    if total_count > num_same_place:
        place_number += 1
        label = 'place' + str(place_number)
        total_count = 1

    src = os.path.join(main_folder, file)

    dst = os.path.join(target_folder, label, folder_name + file)

    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)
    total_count += 1


In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Mon Jun 17 21:36:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   67C    P0              32W /  72W |   8535MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Define your data transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

batch_size = 10

from torch.utils.data import Subset
subset_indices = list(range(1000))

train_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/training_10', transform=transform)
#train_subset = Subset(train_dataset, subset_indices)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/testing_10', transform=transform)
#val_subset = Subset(val_dataset, subset_indices)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

print(len(train_dataset))

class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        #self.fc1 = nn.Linear(30 * 30 * 32, 4096)
        #self.fc2 = nn.Linear(4096, 4096)
        #self.hog_vector = nn.Linear(4096, len_hog_vector)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.pool3(nn.functional.relu(self.conv3(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        #x = nn.functional.relu(self.fc1(x))
        #x = nn.functional.relu(self.fc2(x))
        #x = self.hog_vector(x)
        return x

len_hog_vector = 1764
hogNet = HogNet(len_hog_vector)

hogNet.load_state_dict(torch.load("drive/MyDrive/Thesis/CIFAR_HOG/hog_3Conv3Pool_1764_v2.pth"), strict=False)
#hogNet.load_state_dict(torch.load("drive/MyDrive/Thesis/CIFAR_HOG/hog_3Conv3Pool_1764_v4.pth"))

for param in hogNet.parameters():
    param.requires_grad = False

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        #self.fc1 = nn.Linear(28800, 512)
        self.output = nn.Linear(28800, 800) # 800 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x

# Create an instance of the extended model
nordlandClassifier = NordlandClassifier(hogNet)
nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_3Conv_3Pool_1764_Nordland_Classifier_10_images.pth"))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier.to(device)

from torchsummary import summary
summary(nordlandClassifier, (3, 256, 256))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nordlandClassifier.parameters(), lr=0.001)

# Training loop
num_epochs = 50  # Adjust as needed
old_val_accuracy = 0
for epoch in range(num_epochs):
    nordlandClassifier.train()
    total_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = nordlandClassifier(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation loop
    nordlandClassifier.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = nordlandClassifier(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valdaition_accuracy = correct / total
    if old_val_accuracy < valdaition_accuracy:
      torch.save(nordlandClassifier.state_dict(), 'drive/MyDrive/Thesis/Nordland/HogNet_3Conv_3Pool_1764_Nordland_Classifier_10_images_v2.pth')
      old_val_accuracy = valdaition_accuracy

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss * batch_size / len(train_dataset)}, Validation Accuracy: {100 * valdaition_accuracy}%')

24000
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 254, 254]             896
         MaxPool2d-2         [-1, 32, 127, 127]               0
            Conv2d-3         [-1, 32, 125, 125]           9,248
         MaxPool2d-4           [-1, 32, 62, 62]               0
            Conv2d-5           [-1, 32, 60, 60]           9,248
         MaxPool2d-6           [-1, 32, 30, 30]               0
           Flatten-7                [-1, 28800]               0
            HogNet-8                [-1, 28800]               0
            Linear-9                 [-1, 1600]      46,081,600
Total params: 46,100,992
Trainable params: 46,081,600
Non-trainable params: 19,392
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 25.99
Params size (MB): 175.86
Estimated Total Size (MB): 202.60
---------------------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Define your data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 15

#from torch.utils.data import Subset
#subset_indices = list(range(1000))

train_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/training_10', transform=transform)
#train_subset = Subset(train_dataset, subset_indices)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/testing_10', transform=transform)
#val_subset = Subset(val_dataset, subset_indices)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        #self.fc1 = nn.Linear(16 * 53 * 53, 4096)
        #self.fc2 = nn.Linear(4096, len_hog_vector)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        #x = nn.functional.relu(self.fc1(x))
        #x = nn.functional.relu(self.fc2(x))
        return x

len_hog_vector = 441
hogNet = HogNet(len_hog_vector)

hogNet.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/CIFAR_HOG/hog_441_v2.pth"), strict=False)
#hogNet.load_state_dict(torch.load("drive/MyDrive/Thesis/CIFAR_HOG/hog_3Conv3Pool_1764_v4.pth"))

# For stop training for HOGNet layers
for param in hogNet.parameters():
    param.requires_grad = False

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        #self.fc1 = nn.Linear(28800, 512)
        self.output = nn.Linear(44944, 800) # 1600 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x

# Create an instance of the extended model
nordlandClassifier = NordlandClassifier(hogNet)
#nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier_10_images.pth"))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier.to(device)

from torchsummary import summary
summary(nordlandClassifier, (3, 224, 224))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nordlandClassifier.parameters(), lr=0.001)

# Training loop
num_epochs = 100  # Adjust as needed
old_val_accuracy = 0
for epoch in range(num_epochs):
    nordlandClassifier.train()
    total_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = nordlandClassifier(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation loop
    nordlandClassifier.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = nordlandClassifier(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valdaition_accuracy = correct / total
    if old_val_accuracy < valdaition_accuracy:
      torch.save(nordlandClassifier.state_dict(), 'drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier_10_images_v2.pth')
      old_val_accuracy = valdaition_accuracy

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss * batch_size / len(train_dataset)}, Validation Accuracy: {100 * valdaition_accuracy}%')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 220, 220]             456
         MaxPool2d-2          [-1, 6, 110, 110]               0
            Conv2d-3         [-1, 16, 106, 106]           2,416
         MaxPool2d-4           [-1, 16, 53, 53]               0
           Flatten-5                [-1, 44944]               0
            HogNet-6                [-1, 44944]               0
            Linear-7                  [-1, 800]      35,956,000
Total params: 35,958,872
Trainable params: 35,956,000
Non-trainable params: 2,872
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 5.18
Params size (MB): 137.17
Estimated Total Size (MB): 142.92
----------------------------------------------------------------
Epoch 1/100, Loss: 6.567807366847992, Validation Accuracy: 25.25%
Epoch 2/100, Loss: 6.33915

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Define your data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 15

'''
from torch.utils.data import Subset
subset_indices = list(range(1000))
train_subset = Subset(train_dataset, subset_indices)
'''

train_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/training', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/testing', transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        #self.fc1 = nn.Linear(16 * 53 * 53, 4096)
        #self.fc2 = nn.Linear(4096, len_hog_vector)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        #x = nn.functional.relu(self.fc1(x))
        #x = nn.functional.relu(self.fc2(x))
        return x

len_hog_vector = 441
hogNet = HogNet(len_hog_vector)

hogNet.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/CIFAR_HOG/hog_441_v2.pth"), strict=False)
#hogNet.load_state_dict(torch.load("drive/MyDrive/Thesis/CIFAR_HOG/hog_3Conv3Pool_1764_v4.pth"))

'''
for param in hogNet.parameters():
    param.requires_grad = False
'''

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        #self.fc1 = nn.Linear(28800, 512)
        self.output = nn.Linear(44944, 1600) # 1600 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x

# Create an instance of the extended model
nordlandClassifier = NordlandClassifier(hogNet)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier.to(device)

from torchsummary import summary
summary(nordlandClassifier, (3, 224, 224))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nordlandClassifier.parameters(), lr=0.001)

# Training loop
num_epochs = 50  # Adjust as needed
old_val_accuracy = 0
for epoch in range(num_epochs):
    nordlandClassifier.train()
    total_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = nordlandClassifier(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation loop
    nordlandClassifier.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = nordlandClassifier(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valdaition_accuracy = correct / total
    if old_val_accuracy < valdaition_accuracy:
      torch.save(nordlandClassifier.state_dict(), 'drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier_v2.pth')
      old_val_accuracy = valdaition_accuracy

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss * batch_size / len(train_dataset)}, Validation Accuracy: {100 * valdaition_accuracy}%')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 220, 220]             456
         MaxPool2d-2          [-1, 6, 110, 110]               0
            Conv2d-3         [-1, 16, 106, 106]           2,416
         MaxPool2d-4           [-1, 16, 53, 53]               0
           Flatten-5                [-1, 44944]               0
            HogNet-6                [-1, 44944]               0
            Linear-7                 [-1, 1600]      71,912,000
Total params: 71,914,872
Trainable params: 71,914,872
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 5.18
Params size (MB): 274.33
Estimated Total Size (MB): 280.09
----------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision import models

# Define your data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 15

'''
def get_submodel(model, end_layer_idx):
    # Get the layers up to the specified layer index
    layers = list(model.features.children())[:end_layer_idx + 1]
    return nn.Sequential(*layers)
'''

vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
print(vgg16)

#submodel = get_submodel(vgg16, 23)
#del vgg16.features[23:30]
vgg16.avgpool = nn.Flatten()
vgg16.classifier = nn.Sequential(
    #nn.Linear(100352, 800),
    nn.Linear(25088, 800),
    nn.Softmax(dim=1)
)

submodel = vgg16
print(submodel)

#from torch.utils.data import Subset
#subset_indices = list(range(1000))

train_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/training_10', transform=transform)
#train_subset = Subset(train_dataset, subset_indices)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/testing_10', transform=transform)
#val_subset = Subset(val_dataset, subset_indices)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

for param in submodel.features.parameters():
    param.requires_grad = False

'''
class NordlandClassifier(nn.Module):
    def __init__(self, submodel):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.submodel = submodel
        #self.fc1 = nn.Linear(28800, 512)
        self.flatten = nn.Flatten()
        self.output = nn.Linear(100352, 800) # 1600 place (class)

    def forward(self, x):
        x = self.submodel(x)
        x = self.flatten(x)
        x = nn.functional.softmax(self.output(x), dim=1)
        return x
'''

nordlandClassifier = submodel
# Create an instance of the extended model
#nordlandClassifier = NordlandClassifier(submodel)
#nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier_10_images.pth"))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier.to(device)

from torchsummary import summary
summary(nordlandClassifier, (3, 224, 224))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nordlandClassifier.parameters(), lr=0.001)

# Training loop
num_epochs = 50  # Adjust as needed
old_val_accuracy = 0
for epoch in range(num_epochs):
    nordlandClassifier.train()
    total_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = nordlandClassifier(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation loop
    nordlandClassifier.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = nordlandClassifier(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valdaition_accuracy = correct / total
    if old_val_accuracy < valdaition_accuracy:
      torch.save(nordlandClassifier.state_dict(), 'drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier_10_images.pth')
      old_val_accuracy = valdaition_accuracy

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss * batch_size / len(train_dataset)}, Validation Accuracy: {100 * valdaition_accuracy}%')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense

# Load pre-trained VGG16 model (excluding the top fully-connected layers)
vgg16_model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
#base_model.summary()

import keras.models as models
vpr_model = Model(inputs=vgg16_model.input, outputs=Flatten()(vgg16_model.get_layer('block4_pool').output))

for layer in vpr_model.layers:
   layer.trainable = False

# Check the summary of the modified model
vpr_model.summary()
model = Sequential()
model.add(vpr_model)
model.add(Dense(800, activation='softmax'))
model.summary()

batch_size = 30

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/training_10',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True
                                                 )

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier_10_images.hdf5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

model.fit(training_set, epochs=50, validation_data=test_set, callbacks=[checkpoint])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


800/800 [==============================] - 171s 206ms/step - loss: 2829.8091 - accuracy: 0.4960 - val_loss: 2011.9905 - val_accuracy: 0.6335
Epoch 2/50
800/800 [==============================] - ETA: 0s - loss: 526.1545 - accuracy: 0.8913
Epoch 2: val_accuracy improved from 0.63350 to 0.66475, saving model to drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier_10_images.hdf5
800/800 [==============================] - 166s 208ms/step - loss: 526.1545 - accuracy: 0.8913 - val_loss: 2166.9502 - val_accuracy: 0.6647
Epoch 3/50
800/800 [==============================] - ETA: 0s - loss: 416.9727 - accuracy: 0.9228
Epoch 3: val_accuracy did not improve from 0.66475
800/800 [==============================] - 161s 201ms/step - loss: 416.9727 - accuracy: 0.9228 - val_loss: 2707.0078 - val_accuracy: 0.6440
Epoch 4/50
800/800 [==============================] - ETA: 0s - loss: 319.2112 - accuracy: 0.9415
Epoch 4: val_accuracy improved from 0.66475 to 0.69112, saving model to drive/MyDrive

In [ ]:
from skimage import color
from skimage.feature import hog
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

# Function to preprocess images and extract HOG features
def preprocess_and_extract_hog(directory, target_size, batch_size):
    datagen = ImageDataGenerator()
    generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    num_samples = generator.samples
    num_classes = generator.num_classes
    data = []
    labels = []
    for i in range((num_samples // batch_size) + 1):  # Including the last incomplete batch
        batch_data, batch_labels = next(generator)
        for j in range(len(batch_data)):
            img = batch_data[j]
            gray_image = color.rgb2gray(img)
            hog_feature = hog(gray_image, orientations=9, pixels_per_cell=(32, 32), cells_per_block=(2, 2))
            data.append(hog_feature)
            labels.append(batch_labels[j])
    data = np.array(data)
    labels = np.array(labels)
    return data, labels, num_classes

# Define paths and parameters
train_dir = 'drive/MyDrive/Thesis/Nordland/training'
test_dir = 'drive/MyDrive/Thesis/Nordland/testing'
target_size = (256, 256)
batch_size = 15

# Preprocess training and testing data
x_train, y_train, num_classes_train = preprocess_and_extract_hog(train_dir, target_size, batch_size)
x_test, y_test, num_classes_test = preprocess_and_extract_hog(test_dir, target_size, batch_size)

# Ensure that the number of classes is consistent
num_classes = max(num_classes_train, num_classes_test)

# One-hot encode labels if not already
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

from keras.models import Sequential
from keras.layers import Dense

# Define the model
model = Sequential()
#model.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='drive/MyDrive/Thesis/Nordland/hog_Nordland_Classifier.hdf5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[checkpoint])

Found 24000 images belonging to 1600 classes.
Found 8000 images belonging to 1600 classes.


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import time
import cv2
import os

# Define a simple model
class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        #self.fc1 = nn.Linear(30 * 30 * 32, 4096)
        #self.fc2 = nn.Linear(4096, 4096)
        #self.hog_vector = nn.Linear(4096, len_hog_vector)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.pool3(nn.functional.relu(self.conv3(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        #x = nn.functional.relu(self.fc1(x))
        #x = nn.functional.relu(self.fc2(x))
        #x = self.hog_vector(x)
        return x

len_hog_vector = 1764
hogNet = HogNet(len_hog_vector)

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        #self.fc1 = nn.Linear(28800, 512)
        self.output = nn.Linear(28800, 1600) # 1600 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x


# Create the model and move it to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier = NordlandClassifier(hogNet).to(device)
nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_3Conv_3Pool_1764_Nordland_Classifier.pth"))

# Generate some random input data and move it to the same device as the model
def numerical_sort(value):
    parts = value.split('.')
    return int(parts[0])

reference_folder = 'drive/MyDrive/Thesis/Nordland/Spring'

files = os.listdir(reference_folder)
sorted_files = sorted(files, key=numerical_sort)
file = sorted_files[7000]
query_image = os.path.join(reference_folder, file)

tensor_transform = transforms.Compose([transforms.ToTensor()])
image = cv2.imread(query_image, cv2.IMREAD_COLOR)
resized_img = cv2.resize(image, (256, 256))
resized_img = tensor_transform(resized_img).view(1,3,256,256).to(device)

# Ensure the model is in evaluation mode
nordlandClassifier.eval()

# Measure the forward pass time
torch.cuda.synchronize()  # Wait for GPU operations to finish
start_time = time.time()
with torch.no_grad():
  output = nordlandClassifier(resized_img)
#with torch.no_grad():
  #outputs = nordlandClassifier(resized_img)
  #_, predicted = torch.max(outputs.data, 1)
  #total += labels.size(0)
  #print(predicted)

torch.cuda.synchronize()  # Wait for GPU operations to finish
end_time = time.time()

# Calculate the elapsed time
forward_pass_time = end_time - start_time
print(f"Forward pass time: {forward_pass_time} seconds")

#softmax = nn.Softmax(dim=1)
#probabilities = softmax(output)

# Get the predicted class
print(len(output))
predicted_class = torch.argmax(output, dim=1).item()
print(f"Predicted class: {predicted_class}")

Forward pass time: 0.002850770950317383 seconds
1
Predicted class: 418


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import time
import cv2
import os

class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        #self.fc1 = nn.Linear(16 * 53 * 53, 4096)
        #self.fc2 = nn.Linear(4096, len_hog_vector)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        #x = nn.functional.relu(self.fc1(x))
        #x = nn.functional.relu(self.fc2(x))
        return x

len_hog_vector = 441
hogNet = HogNet(len_hog_vector)

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        self.output = nn.Linear(44944, 1600) # 1600 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x


# Create the model and move it to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier = NordlandClassifier(hogNet).to(device)
nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier.pth"))

# Generate some random input data and move it to the same device as the model
def numerical_sort(value):
    parts = value.split('.')
    return int(parts[0])

reference_folder = 'drive/MyDrive/Thesis/Nordland/Spring'

files = os.listdir(reference_folder)
sorted_files = sorted(files, key=numerical_sort)
file = sorted_files[6000]
query_image = os.path.join(reference_folder, file)

tensor_transform = transforms.Compose([transforms.ToTensor()])
image = cv2.imread(query_image, cv2.IMREAD_COLOR)
resized_img = cv2.resize(image, (224, 224))
resized_img = tensor_transform(resized_img).view(1,3,224,224).to(device)

# Ensure the model is in evaluation mode
nordlandClassifier.eval()

# Measure the forward pass time
torch.cuda.synchronize()  # Wait for GPU operations to finish
start_time = time.time()
with torch.no_grad():
  output = nordlandClassifier(resized_img)
#with torch.no_grad():
  #outputs = nordlandClassifier(resized_img)
  #_, predicted = torch.max(outputs.data, 1)
  #total += labels.size(0)
  #print(predicted)
  #correct += (predicted == labels).sum().item()

torch.cuda.synchronize()  # Wait for GPU operations to finish
end_time = time.time()

# Calculate the elapsed time
forward_pass_time = end_time - start_time
print(f"Forward pass time: {forward_pass_time} seconds")

#softmax = nn.Softmax(dim=1)
#probabilities = softmax(output)

# Get the predicted class
#print(output[0][669])
#print(output[0][1599])
predicted_class = torch.argmax(output, dim=1).item()
print(f"Predicted class: {predicted_class}")

Forward pass time: 0.0033266544342041016 seconds
Predicted class: 226


In [ ]:
import os
import keras.models as models
import cv2
import tensorflow as tf
import time

vgg16_model = models.load_model("drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier.hdf5")

def numerical_sort(value):
    parts = value.split('.')
    return int(parts[0])

reference_folder = 'drive/MyDrive/Thesis/Nordland/Spring'

files = os.listdir(reference_folder)
sorted_files = sorted(files, key=numerical_sort)
file = sorted_files[2500]
query_image = os.path.join(reference_folder, file)

img = cv2.imread(query_image, cv2.IMREAD_COLOR)
resized_img = tf.image.resize(img, (224, 224))
input_set = [resized_img]
input_set = tf.convert_to_tensor(input_set, dtype=tf.int16)
start_time = time.time()
output = vgg16_model.predict(input_set)
end_time = time.time()

forward_pass_time = end_time - start_time
print(f"Forward pass time using model.predict: {forward_pass_time} seconds")

print(output[0][13])
predicted_class = tf.argmax(output, axis=-1).numpy()[0]
print(f"Predicted class: {predicted_class}")

1/1 [==============================] - 0s 124ms/step
Forward pass time using model.predict: 0.17297005653381348 seconds
0.0
Predicted class: 1048


In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 18.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchmetrics.classification import MulticlassRecall

transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

#from torch.utils.data import Subset
#subset_indices = list(range(200))

batch_size = 15

val_dataset = datasets.ImageFolder(root='drive/MyDrive/Thesis/Nordland/testing_10', transform=transform)
#val_subset = Subset(val_dataset, subset_indices)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

class HogNet(nn.Module):
    def __init__(self, len_hog_vector):
        super(HogNet, self).__init__()
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.flatten(x)
        return x

len_hog_vector = 441
hogNet = HogNet(len_hog_vector)

hogNet.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/CIFAR_HOG/hog_441_v2.pth"), strict=False)

for param in hogNet.parameters():
    param.requires_grad = False

class NordlandClassifier(nn.Module):
    def __init__(self, hogNet):
        super(NordlandClassifier, self).__init__()
        # Copy layers from the original model
        self.hogNet = hogNet
        #self.fc1 = nn.Linear(28800, 512)
        self.output = nn.Linear(44944, 800) # 1600 place (class)

    def forward(self, x):
        x = self.hogNet(x)
        #x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.output(x), dim=1)
        return x

# Create an instance of the extended model
nordlandClassifier = NordlandClassifier(hogNet)
nordlandClassifier.load_state_dict(torch.load("drive/MyDrive/Thesis/Nordland/HogNet_2Conv_2Pool_441_Nordland_Classifier_10_images.pth"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nordlandClassifier.to(device)


# Define function to calculate Recall@k
def calculate_recall_at_k():
  nordlandClassifier.eval()

  metric_1 = MulticlassRecall(num_classes=800, average='micro', top_k=1)
  metric_1.to(device)
  metric_2 = MulticlassRecall(num_classes=800, average='micro', top_k=2)
  metric_2.to(device)
  metric_3 = MulticlassRecall(num_classes=800, average='micro', top_k=3)
  metric_3.to(device)
  metric_4 = MulticlassRecall(num_classes=800, average='micro', top_k=4)
  metric_4.to(device)
  metric_5 = MulticlassRecall(num_classes=800, average='micro', top_k=5)
  metric_5.to(device)
  metric_10 = MulticlassRecall(num_classes=800, average='micro', top_k=10)
  metric_10.to(device)
  metric_20 = MulticlassRecall(num_classes=800, average='micro', top_k=20)
  metric_20.to(device)

  all_preds = []
  all_targets = []
  correct = 0
  total = 0

  with torch.no_grad():  # Disable gradient calculation for efficiency
    for data, targets in val_dataloader:
      data, targets = data.to(device), targets.to(device)
      outputs = nordlandClassifier(data)
      _, predicted = torch.max(outputs.data, 1)
      total += outputs.size(0)
      correct += (predicted == targets).sum().item()

      all_preds.append(outputs)
      all_targets.append(targets)

  valdaition_accuracy = correct / total
  print(f'Validation Accuracy: {100 * valdaition_accuracy}%')

  recall_1 = 0
  recall_2 = 0
  recall_3 = 0
  recall_4 = 0
  recall_5 = 0
  recall_10 = 0
  recall_20 = 0

  for i, pred in enumerate(all_preds):
    recall_1 += metric_1(all_preds[i], all_targets[i])

  recall_1 = recall_1 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_2 += metric_2(all_preds[i], all_targets[i])

  recall_2 = recall_2 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_3 += metric_3(all_preds[i], all_targets[i])

  recall_3 = recall_3 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_4 += metric_4(all_preds[i], all_targets[i])

  recall_4 = recall_4 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_5 += metric_5(all_preds[i], all_targets[i])

  recall_5 = recall_5 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_10 += metric_10(all_preds[i], all_targets[i])

  recall_10 = recall_10 / len(all_preds)

  for i, pred in enumerate(all_preds):
    recall_20 += metric_20(all_preds[i], all_targets[i])

  recall_20 = recall_20 / len(all_preds)

  return recall_1.item(), recall_2.item(), recall_3.item(), recall_4.item(), recall_5.item(), recall_10.item(), recall_20.item()   # Get and return average recall

recall_at_1, recall_at_2, recall_at_3, recall_at_4, recall_at_5, recall_at_10, recall_at_20 = calculate_recall_at_k()
print(f"Recall@1: {recall_at_1}")

print(f"Recall@2: {recall_at_2}")

print(f"Recall@3: {recall_at_3}")

print(f"Recall@4: {recall_at_4}")

print(f"Recall@5: {recall_at_5}")

print(f"Recall@10: {recall_at_10}")

print(f"Recall@20: {recall_at_20}")

Validation Accuracy: 71.3625%
Recall@1: 0.7134836316108704
Recall@2: 0.7821465134620667
Recall@3: 0.8116095066070557
Recall@4: 0.8295868635177612
Recall@5: 0.8419461846351624
Recall@10: 0.8772764801979065
Recall@20: 0.9076131582260132


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
import keras.models as models

vgg16_model = models.load_model("drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier_10_images.hdf5")

# Function to calculate recall at k
def recall_at_k(y_true, y_pred, k):
    """
    Calculates Recall at K for the given true labels and predictions.

    Args:
    y_true: Tensor of true labels (shape: [batch_size])
    y_pred: Tensor of predicted probabilities (shape: [batch_size, num_classes])
    k: Integer, top k predictions to consider for recall

    Returns:
    recall_at_k: Tensor containing recall at k value
    """
    # Get the top k predictions
    top_k_preds = tf.math.top_k(y_pred, k=k).indices

    # Expand the true labels to match the shape of top_k_preds
    y_true_expanded = tf.expand_dims(y_true, axis=-1)

    # Check if the true label is in the top k predictions
    correct_preds = tf.reduce_any(tf.equal(y_true_expanded, top_k_preds), axis=-1)

    # Calculate recall at k
    recall_at_k = tf.reduce_mean(tf.cast(correct_preds, dtype=tf.float32))

    return recall_at_k

# Data generators
batch_size = 30

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Getting predictions and true labels
y_true = []
y_pred = []

for i in range(len(test_set)):
    x_batch, y_batch = test_set[i]
    y_true_batch = tf.argmax(y_batch, axis=1)  # Get the true labels
    y_pred_batch = vgg16_model.predict(x_batch)  # Get the predicted probabilities

    y_true.extend(y_true_batch.numpy())
    y_pred.extend(y_pred_batch)

y_true = tf.constant(y_true)
y_pred = tf.constant(y_pred)

# Calculate Recall@1
recall_at_1 = recall_at_k(y_true, y_pred, k=1)
print("Recall@1:", recall_at_1.numpy())

# Calculate Recall@2
recall_at_2 = recall_at_k(y_true, y_pred, k=2)
print("Recall@2:", recall_at_2.numpy())

# Calculate Recall@3
recall_at_3 = recall_at_k(y_true, y_pred, k=3)
print("Recall@3:", recall_at_3.numpy())

# Calculate Recall@4
recall_at_4 = recall_at_k(y_true, y_pred, k=4)
print("Recall@4:", recall_at_4.numpy())

# Calculate Recall@5
recall_at_5 = recall_at_k(y_true, y_pred, k=5)
print("Recall@5:", recall_at_5.numpy())

# Calculate Recall@10
recall_at_10 = recall_at_k(y_true, y_pred, k=10)
print("Recall@10:", recall_at_10.numpy())

# Calculate Recall@20
recall_at_20 = recall_at_k(y_true, y_pred, k=20)
print("Recall@20:", recall_at_20.numpy())

Found 8000 images belonging to 800 classes.
1/1 [==============================] - 0s 26ms/step
Recall@1: 0.87775
Recall@2: 0.878375
Recall@3: 0.878375
Recall@4: 0.878375
Recall@5: 0.878375
Recall@10: 0.87925
Recall@20: 0.88


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
import keras.models as models

vgg16_model = models.load_model("drive/MyDrive/Thesis/Nordland/vgg16_pool4_Nordland_Classifier_10_images.hdf5")

# Function to calculate recall at k
def recall_at_k(y_true, y_pred, k):
    """
    Calculates Recall at K for the given true labels and predictions.

    Args:
    y_true: Tensor of true labels (shape: [batch_size])
    y_pred: Tensor of predicted probabilities (shape: [batch_size, num_classes])
    k: Integer, top k predictions to consider for recall

    Returns:
    recall_at_k: Tensor containing recall at k value
    """
    # Get the top k predictions
    top_k_preds = tf.math.top_k(y_pred, k=k).indices

    # Expand the true labels to match the shape of top_k_preds
    y_true_expanded = tf.expand_dims(y_true, axis=-1)

    # Check if the true label is in the top k predictions
    correct_preds = tf.reduce_any(tf.equal(y_true_expanded, top_k_preds), axis=-1)

    # Calculate recall at k
    recall_at_k = tf.reduce_mean(tf.cast(correct_preds, dtype=tf.float32))

    return recall_at_k

# Data generators
batch_size = 30

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Initialize the metric
recall_micro = tf.keras.metrics.Recall(top_k=1)

# Getting predictions and true labels
for i in range(len(test_set)):
    x_batch, y_batch = test_set[i]
    y_true_batch = tf.argmax(y_batch, axis=1)  # Get the true labels
    y_pred_batch = vgg16_model.predict(x_batch)  # Get the predicted probabilities

    # Update the recall metric with the current batch
    recall_micro.update_state(y_true_batch, y_pred_batch)

# Calculate Recall@1
recall_at_1 = recall_micro.result().numpy()
print("Recall@1 (micro):", recall_at_1)

# To calculate Recall@5
recall_micro_top5 = tf.keras.metrics.Recall(top_k=5)

# Reset and calculate Recall@5
for i in range(len(test_set)):
    x_batch, y_batch = test_set[i]
    y_true_batch = tf.argmax(y_batch, axis=1)  # Get the true labels
    y_pred_batch = model.predict(x_batch)  # Get the predicted probabilities

    # Update the recall metric with the current batch
    recall_micro_top5.update_state(y_true_batch, y_pred_batch)

recall_at_5 = recall_micro_top5.result().numpy()
print("Recall@5 (micro):", recall_at_5)

Found 8000 images belonging to 800 classes.
1/1 [==============================] - 6s 6s/step


ValueError: Shapes (30, 800) and (30,) are incompatible

In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense

# Load pre-trained VGG16 model (excluding the top fully-connected layers)
resnet_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#base_model.summary()
# include_top=False To exclude the classification part of model and get just features extracting part of model
import keras.models as models
vpr_model = resnet_model

for layer in vpr_model.layers:
   layer.trainable = False

# Check the summary of the modified model
vpr_model.summary()

model = Sequential()
model.add(vpr_model)
model.add(Flatten())
model.add(Dense(800, activation='softmax'))
model.summary()

batch_size = 30

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/training_10',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True
                                                 )

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'drive/MyDrive/Thesis/Nordland/resnet100_Nordland_Classifier_10_images.hdf5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

model.fit(training_set, epochs=50, validation_data=test_set, callbacks=[checkpoint])

171446536/171446536 [==============================] - 1s 0us/step
Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


800/800 [==============================] - 9495s 12s/step - loss: 62.9149 - accuracy: 0.3825 - val_loss: 83.0298 - val_accuracy: 0.3130
Epoch 2/50
800/800 [==============================] - ETA: 0s - loss: 13.4280 - accuracy: 0.8253
Epoch 2: val_accuracy improved from 0.31300 to 0.39625, saving model to drive/MyDrive/Thesis/Nordland/resnet100_Nordland_Classifier_10_images.hdf5
800/800 [==============================] - 165s 207ms/step - loss: 13.4280 - accuracy: 0.8253 - val_loss: 80.3068 - val_accuracy: 0.3963
Epoch 3/50
800/800 [==============================] - ETA: 0s - loss: 9.1173 - accuracy: 0.8851
Epoch 3: val_accuracy did not improve from 0.39625
800/800 [==============================] - 156s 190ms/step - loss: 9.1173 - accuracy: 0.8851 - val_loss: 111.0988 - val_accuracy: 0.3475
Epoch 4/50
800/800 [==============================] - ETA: 0s - loss: 8.5985 - accuracy: 0.9022
Epoch 4: val_accuracy did not improve from 0.39625
800/800 [==============================] - 153s 192m

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense

# Load pre-trained VGG16 model (excluding the top fully-connected layers)
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#base_model.summary()
# weights='imagenet' To use model with weights that are gotten after training on imageNet dataset
import keras.models as models
vpr_model = mobilenet_model

for layer in vpr_model.layers:
   layer.trainable = False

# Check the summary of the modified model
vpr_model.summary()

model = Sequential()
model.add(vpr_model)
model.add(Flatten())
model.add(Dense(800, activation='softmax'))
model.summary()

batch_size = 30

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/training_10',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True
                                                 )

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

#monitor = 'val_accuracy' For regression tasks we should use 'val_loss'
#mode = max For accuarcy but it should equal 'min' for loss


model.fit(training_set, epochs=50, validation_data=test_set, callbacks=[checkpoint])

17225924/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


800/800 [==============================] - 154s 191ms/step - loss: 38.0960 - accuracy: 0.2517 - val_loss: 55.1684 - val_accuracy: 0.1287
Epoch 2/50
800/800 [==============================] - ETA: 0s - loss: 9.4065 - accuracy: 0.7207
Epoch 2: val_accuracy improved from 0.12875 to 0.15875, saving model to drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5
800/800 [==============================] - 156s 195ms/step - loss: 9.4065 - accuracy: 0.7207 - val_loss: 56.0449 - val_accuracy: 0.1587
Epoch 3/50
800/800 [==============================] - ETA: 0s - loss: 6.7263 - accuracy: 0.8083
Epoch 3: val_accuracy did not improve from 0.15875
800/800 [==============================] - 151s 188ms/step - loss: 6.7263 - accuracy: 0.8083 - val_loss: 68.6636 - val_accuracy: 0.1534
Epoch 4/50
800/800 [==============================] - ETA: 0s - loss: 6.1068 - accuracy: 0.8403
Epoch 4: val_accuracy did not improve from 0.15875
800/800 [==============================] - 150s 188ms/

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
import keras.models as models

resnet101_model = models.load_model("drive/MyDrive/Thesis/Nordland/resnet100_Nordland_Classifier_10_images.hdf5")

# Function to calculate recall at k
def recall_at_k(y_true, y_pred, k):
    """
    Calculates Recall at K for the given true labels and predictions.

    Args:
    y_true: Tensor of true labels (shape: [batch_size])
    y_pred: Tensor of predicted probabilities (shape: [batch_size, num_classes])
    k: Integer, top k predictions to consider for recall

    Returns:
    recall_at_k: Tensor containing recall at k value
    """
    # Get the top k predictions
    top_k_preds = tf.math.top_k(y_pred, k=k).indices

    # Expand the true labels to match the shape of top_k_preds
    y_true_expanded = tf.expand_dims(y_true, axis=-1)

    # Check if the true label is in the top k predictions
    correct_preds = tf.reduce_any(tf.equal(y_true_expanded, top_k_preds), axis=-1)

    # Calculate recall at k
    recall_at_k = tf.reduce_mean(tf.cast(correct_preds, dtype=tf.float32))

    return recall_at_k

# Data generators
batch_size = 30

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Getting predictions and true labels
y_true = []
y_pred = []

for i in range(len(test_set)):
    x_batch, y_batch = test_set[i]
    y_true_batch = tf.argmax(y_batch, axis=1)  # Get the true labels
    y_pred_batch = resnet101_model.predict(x_batch)  # Get the predicted probabilities

    y_true.extend(y_true_batch.numpy())
    y_pred.extend(y_pred_batch)

y_true = tf.constant(y_true)
y_pred = tf.constant(y_pred)

# Calculate Recall@1
recall_at_1 = recall_at_k(y_true, y_pred, k=1)
print("Recall@1:", recall_at_1.numpy())

# Calculate Recall@2
recall_at_2 = recall_at_k(y_true, y_pred, k=2)
print("Recall@2:", recall_at_2.numpy())

# Calculate Recall@3
recall_at_3 = recall_at_k(y_true, y_pred, k=3)
print("Recall@3:", recall_at_3.numpy())

# Calculate Recall@4
recall_at_4 = recall_at_k(y_true, y_pred, k=4)
print("Recall@4:", recall_at_4.numpy())

# Calculate Recall@5
recall_at_5 = recall_at_k(y_true, y_pred, k=5)
print("Recall@5:", recall_at_5.numpy())

# Calculate Recall@10
recall_at_10 = recall_at_k(y_true, y_pred, k=10)
print("Recall@10:", recall_at_10.numpy())

# Calculate Recall@20
recall_at_20 = recall_at_k(y_true, y_pred, k=20)
print("Recall@20:", recall_at_20.numpy())

Found 8000 images belonging to 800 classes.
1/1 [==============================] - 1s 535ms/step
Recall@1: 0.54125
Recall@2: 0.59925
Recall@3: 0.61575
Recall@4: 0.620625
Recall@5: 0.622
Recall@10: 0.625
Recall@20: 0.627


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
import keras.models as models

mobilenet_model = models.load_model("drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5")

# Function to calculate recall at k
def recall_at_k(y_true, y_pred, k):
    """
    Calculates Recall at K for the given true labels and predictions.

    Args:
    y_true: Tensor of true labels (shape: [batch_size])
    y_pred: Tensor of predicted probabilities (shape: [batch_size, num_classes])
    k: Integer, top k predictions to consider for recall

    Returns:
    recall_at_k: Tensor containing recall at k value
    """
    # Get the top k predictions
    top_k_preds = tf.math.top_k(y_pred, k=k).indices

    # Expand the true labels to match the shape of top_k_preds
    y_true_expanded = tf.expand_dims(y_true, axis=-1)

    # Check if the true label is in the top k predictions
    correct_preds = tf.reduce_any(tf.equal(y_true_expanded, top_k_preds), axis=-1)

    # Calculate recall at k
    recall_at_k = tf.reduce_mean(tf.cast(correct_preds, dtype=tf.float32))

    return recall_at_k

# Data generators
batch_size = 30

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory('drive/MyDrive/Thesis/Nordland/testing_10',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True
                                            )

# Getting predictions and true labels
y_true = []
y_pred = []

for i in range(len(test_set)):
    x_batch, y_batch = test_set[i]
    y_true_batch = tf.argmax(y_batch, axis=1)  # Get the true labels
    y_pred_batch = mobilenet_model.predict(x_batch)  # Get the predicted probabilities

    y_true.extend(y_true_batch.numpy())
    y_pred.extend(y_pred_batch)

y_true = tf.constant(y_true)
y_pred = tf.constant(y_pred)

# Calculate Recall@1
recall_at_1 = recall_at_k(y_true, y_pred, k=1)
print("Recall@1:", recall_at_1.numpy())

# Calculate Recall@2
recall_at_2 = recall_at_k(y_true, y_pred, k=2)
print("Recall@2:", recall_at_2.numpy())

# Calculate Recall@3
recall_at_3 = recall_at_k(y_true, y_pred, k=3)
print("Recall@3:", recall_at_3.numpy())

# Calculate Recall@4
recall_at_4 = recall_at_k(y_true, y_pred, k=4)
print("Recall@4:", recall_at_4.numpy())

# Calculate Recall@5
recall_at_5 = recall_at_k(y_true, y_pred, k=5)
print("Recall@5:", recall_at_5.numpy())

# Calculate Recall@10
recall_at_10 = recall_at_k(y_true, y_pred, k=10)
print("Recall@10:", recall_at_10.numpy())

# Calculate Recall@20
recall_at_20 = recall_at_k(y_true, y_pred, k=20)
print("Recall@20:", recall_at_20.numpy())

Found 8000 images belonging to 800 classes.
1/1 [==============================] - 0s 87ms/step
Recall@1: 0.229875
Recall@2: 0.30025
Recall@3: 0.33425
Recall@4: 0.357
Recall@5: 0.37075
Recall@10: 0.397625
Recall@20: 0.406875


In [ ]:
# prompt: I want to train vgg16 on dataset with 50 epochs

# Load pre-trained VGG16 model (excluding the top fully-connected layers)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = vgg16_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
predictions = Dense(1000, activation='softmax')(x)

# Create the new model
model = Model(inputs=vgg16_model.input, outputs=predictions)

# Freeze the convolutional layers in the pre-trained model
for layer in vgg16_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your dataset
batch_size = 32
epochs = 50

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs,
    batch_size=batch_size
)


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import keras.models as models

# 1️⃣ Load the saved model
#mobilenet_model = models.load_model("drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5")
mobilenet_model = models.load_model("drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5", compile=False)

# 2️⃣ Define the target image size (MobileNet default = 224x224)
IMG_SIZE = (224, 224)

# 3️⃣ Function to preprocess and predict an image
def predict_image_class(img_path, model, class_names=None):
    # Load image
    img = image.load_img(img_path, target_size=IMG_SIZE)

    # Convert to array
    img_array = image.img_to_array(img)

    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize same as during training
    #img_array = tf.keras.applications.mobilenet.preprocess_input(img_array)

    # Predict
    predictions = model.predict(img_array)

    print("Predictions shape:", predictions.shape)
    print("Predictions:", predictions)

    '''
    # Get class index
    #predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_index = int(np.argmax(predictions.flatten()))


    if hasattr(predicted_class_index, 'item'):
        predicted_class_index = predicted_class_index.item()

    # Map index to class name if provided
    if class_names:
        predicted_class_name = class_names[predicted_class_index]
        print(f"Predicted class: {predicted_class_name} ({predicted_class_index})")
        return predicted_class_name
    else:
        print(f"Predicted class index: {predicted_class_index}")
        return predicted_class_index
   '''
# 4️⃣ Example usage
img_path = "drive/MyDrive/Thesis/Nordland/testing_10/Spring60.png"

# If you know your class names (in the same order used during training):
class_names = [i for i in range(1, 801)]  # Example — edit for your dataset

predict_image_class(img_path, mobilenet_model, class_names)


TypeError: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.layers', 'class_name': 'BatchNormalization', 'config': {'name': 'conv1_bn', 'trainable': False, 'dtype': 'float32', 'axis': [3], 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'gamma_initializer': {'module': 'keras.initializers', 'class_name': 'Ones', 'config': {}, 'registered_name': None}, 'moving_mean_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'moving_variance_initializer': {'module': 'keras.initializers', 'class_name': 'Ones', 'config': {}, 'registered_name

In [22]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.models import load_model
model = load_model("drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images.hdf5", compile=False)
##model.save("drive/MyDrive/Thesis/Nordland/MobileNet_Nordland_Classifier_10_images_tf")  # New format


2.19.0


TypeError: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.layers', 'class_name': 'BatchNormalization', 'config': {'name': 'conv1_bn', 'trainable': False, 'dtype': 'float32', 'axis': [3], 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'gamma_initializer': {'module': 'keras.initializers', 'class_name': 'Ones', 'config': {}, 'registered_name': None}, 'moving_mean_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'moving_variance_initializer': {'module': 'keras.initializers', 'class_name': 'Ones', 'config': {}, 'registered_name